# Parasitic (Zero-lift) drag

This section describes how to compute parasitic (zero-lift) drag coefficient ($C_{D_0}$) using drag buildup method and demonstrates it using the example airplane. The drag buildup method consists of computing zero-lift drag for each major component of the airplane and then summing it up to obtain total zero-lift drag for the entire airplane. The zero lift drag for any component can be divided into two parts: skin friction drag and form drag. The skin friction drag is calculated based on the equivalent flat plate theory. However, in conceptual design phase, it is difficult to get a good estimate of form drag. Hence, a factor (greater than 1) is multiplied to skin friction drag to account for form drag. Once parasitic drag for individual components are evaluated, they are added to obtain final value. Additionally, extra terms representing other sources of zero-lift drag can also be added. Refer to lecture notes, section 12.5 in Raymer, section 13.3 in Nicolai and Carichner, and section 15.4.6 in Gudmundsson for more details. Mathematically, the drag buildup method can be written as

$$
    C_{D_0} = \frac{ \sum\limits_{i=1}^n C_{f_i} FF_i Q_i S_{wet_i} }{ S_{ref}} + C_{D_{misc}} + C_{D_{L\&P}},
$$

where $C_f$ is the skin friction coefficient, $FF$ represents form factor to account for form drag, $Q_i$ denotes inteference factor, $S_{wet}$ is the wetted area, and $S_{ref}$ is the reference area. The subscript $i$ denotes $i^{th}$ component and $n$ is the total number of components. The $C_{D_{misc}}$ represents miscellaneous drag caused by flaps, fixed landing gear, fuselage aft upsweep angle, etc. The $C_{D_{L\&P}}$ refers to the drag due to leakages and protuberances.

For the example airplane, four components are considered in clean (cruise) configuration: wing, fuselage, horizontal tail, and vertical tail. Below subsections compute each term in the above equation for each component, and then computes final $C_{D_0}$ for the example airplane.

## Skin friction coefficient

The skin friction coefficient ($C_f$) is computed by assuming flow over a flat plate. The value of $C_f$ is dependent on the type of flow (laminar or turbulent), which in turn is dependent on the Reynolds number ($Re$), given by

$$
    Re = \frac{\rho Vl}{\mu},
$$

where $\rho$ is the density, $V$ is the freestream velocity, $l$ is the characteristic length (typically length of the component in the direction of flow), and $\mu$ is the dynamic viscosity. Based on the $Re$, $C_f$ is computed as (Raymer Eq. 12.26 and 12.27):

$$
    C_f = \frac{1.328}{\sqrt{Re}} \text{ (laminar)} \quad \text{and} \quad C_f = \frac{0.455}{(\log_{10}Re)^{2.58}} \text{ (turbulent)} 
$$

Usually, the flow over airplane components is almost turbulent. Table 12.4 in Raymer provides some guidance for estimating what percent of wetted area might be experiencing laminar flow. For the example airplane, it is assumed that fuselage has 10% laminar flow, while all other components have fully turbulent flow. If the flow is mixed, then $C_f$ is computed as a weighted average of laminar and turbulent values.

Note that skin roughness plays an important role in skin friction drag. As recommended by Raymer, a cutoff Re (equation 12.28 in Raymer) can be used to account for the increased turbulent flow. It was determined that the cutoff Re was much large than the actual Re for the example airplane, and hence, not used here. Additionally, $C_f$ for turbulent flow can be corrected for compressibility effects using Raymer equation 12.27, but it was not used since example airplane will be flying at low speeds.

Below code block computes skin friction coefficient for each component:

In [19]:
import math

def compute_re_cruise(l):
    """
        Function to compute Re, l should be in m
    """

    rho = 0.962870 # kg/m^3, 8000 ft
    mu = 0.0000173188 # Pa.s, 8000 ft
    V = 102.89 # m/s

    return rho*V*l/mu

def compute_cf_turbulent(re):
    """
        Function to compute Cf for turbulent flow
    """

    return 0.455/math.log10(re)**2.58

def compute_cf_laminar(re):
    """
        Function to compute Cf for laminar flow
    """

    return 1.328/re**0.5

# Compute Re
fuselage_re = compute_re_cruise(31.6*0.3048) # 31.6 ft to m
wing_re = compute_re_cruise(4.3*0.3048) # 4.3 ft to m
ht_re = compute_re_cruise(2.4*0.3048) # 2.4 ft to m
vt_re = compute_re_cruise(3.7*0.3048) # 3.7 ft to m

# Compute fuselage Cf
Cf_fuselage_laminar = compute_cf_laminar(fuselage_re)
Cf_fuselage_turbulent = compute_cf_turbulent(fuselage_re)
Cf_fuselage = Cf_fuselage_laminar*0.1 + Cf_fuselage_turbulent*0.9
print(f"Cf fuselage: {Cf_fuselage:.6f}")

# Compute wing Cf
Cf_wing = compute_cf_turbulent(wing_re)
print(f"Cf wing: {Cf_wing:.6f}")

# Compute ht Cf
Cf_ht = compute_cf_turbulent(ht_re)
print(f"Cf ht: {Cf_ht:.6f}")

# Compute vt Cf
Cf_vt = compute_cf_turbulent(vt_re)
print(f"Cf vt: {Cf_vt:.6f}")

Cf fuselage: 0.002103
Cf wing: 0.003147
Cf ht: 0.003467
Cf vt: 0.003225


## Form and Interference factors

Next, form factor ($FF$) for each component is determined. The $FF$ for wings and tails is determined using:

$$
    FF = \bigg[ 1 + \frac{0.6}{(x/c)_m} (t/c) + 100 (t/c)^4 \bigg],
$$

where $(x/c)_m$ is the location of maximum thickness as a fraction of chord, and $t/c$ is the thickness-to-chord ratio.

For the example airplane, $(x/c)_m = 0.3$ for the wing and tails. The $t/c$ is set to 0.18 for the wing and 0.12 for the tails. The form factor computed using above equation can be adjusted for compressibility and sweep effects, but it was not done here since the example airplane will be flying at low speeds and has no sweep. The $FF$ for fuselage is determined using:

$$
    FF = \bigg[ 0.9 + \frac{5}{f^{1.5}} + \frac{f}{400} \bigg],
$$

where $f$ is the fuselage fineness ratio. The interference factor ($Q$) for fuselage and wing is set to 1.0, while it is set to 1.04 for both tails. Refer to section 12.5.4 and 12.5.5 in Raymer for more details about $FF$ and $Q$.

Below code block computes $FF$ for wing, fuselage, horizontal tail, and vertical tail:

In [21]:
compute_ff_fuselage = lambda f: 0.9 + 5/f**1.5 + f/400
compute_ff_wing_tail = lambda xc, tc: 1 + 0.6*tc/xc + 100*tc**4

ff_fuselage = compute_ff_fuselage(5.5)
ff_wing = compute_ff_wing_tail(0.3, 0.18)
ff_tail = compute_ff_wing_tail(0.3, 0.12)

print(f"Fuselage form factor: {ff_fuselage:.6f}")
print(f"Wing form factor: {ff_wing:.6f}")
print(f"Tail form factor: {ff_tail:.6f}")

Fuselage form factor: 1.301388
Wing form factor: 1.464976
Tail form factor: 1.260736


Raymer Section 12.5

Components: Wing, Fuselage, HT, VT

Laminar flow on Fuselage (10%), Wing (5%), Tails (0%)

Reynolds number (+ adjusted for skin surface)

Cf for laminar and turbulent

Type of flow -> Re -> Cf -> FF -> Q -> S_wet -> 

Note about not including LG, Power plant (other than nacelle)

Note about wetted are begin computed from OpenVSP

Fuselage wetted area = 380 ft^2

Wing wetted area = 224 ft^2 (+ 52 )

VT wetted area = 39 ft^2

HT wetted area = 60 fT^2

As outlined in Raymer, initial estimate for $C_{D_0}$ can be computed using

$$
    C_{D_0} = C_{f_e} \frac{S_{wet}}{S_{ref}},
$$

where $C_{f_e}$ is the equivalent skin friction coefficient, $S_{wet}$ is total airplane wetted area, and $S_{ref}$ is the reference area. The $C_{f_e}$ is obtained from Table 12.3 and $S_{wet}$ can be easily obtained from OpenVSP model. The value obtained from this estimation can be used as a sanity check for the detailed drag buildup method.